## GEE with Python

conda create -n gee python=3.8

conda activate gee

pip install earthengine-api geemap windows-curses

conda install jupyter

conda install ipykernel

python -m ipykernel install --user --name gee --display-name "Python (gee)"

jupyter notebook

In [1]:
import ee
import geemap

# Authentifier et initialiser Google Earth Engine
ee.Authenticate()
ee.Initialize()

## Importation des données raster

In [2]:
# Fonction pour obtenir les images raster par année et sélectionner la première bande
def importation_rasters(annee):
    nom_raster = f"202406_Global_Pf_Parasite_Rate_BFA_{annee}"
    asset_id = f'projects/ee-khadijaaatou180702/assets/{nom_raster}'
    return ee.Image(asset_id).select(0)

# Liste des années pour les rasters
annees = list(range(2000, 2023))

# Charger toutes les images raster et sélectionner la première bande
rasters = [importation_rasters(annee) for annee in annees]

# Créer une collection à partir de la liste d'images
stack_raster = ee.ImageCollection(rasters)

## Cherchons le raster portant les taux moyens de parasite entre 2000 et 2022 pour chaque pixel

In [3]:
# Calculons la moyenne sur la collection
moyenne_raster = stack_raster.mean()

In [4]:
from ipyleaflet import ImageOverlay
# Définissons les paramètres de visualisation
vis_params = {
    'min': 0,
    'max': 1,
    'palette': ['blue', 'green', 'yellow', 'red']
}
 
# Créer une instance de carte unique
Map = geemap.Map(center=[0, 0], zoom=2)

# Fonction pour visualiser une image en ajoutant une couche à la carte
def visualize_image(image, vis_params, title):
    Map.addLayer(image, title, vis_params)

# Visualiser chaque statistique
visualize_image(moyenne_raster, 'moyenne_raster' , vis_params)


# Ajouter une barre de couleurs (légende automatique)
Map.add_colorbar(vis_params=vis_params, label="Valeur", layer_name='moyenne_raster')

# Ajouter une flèche du nord
north_arrow_url = 'https://upload.wikimedia.org/wikipedia/commons/e/eb/North_arrow.svg'  # URL de l'image de la flèche du nord
north_arrow = ImageOverlay(
    url=north_arrow_url,
    bounds=[[0, 0], [0.1, 0.1]],  # Ajuster les coordonnées pour placer la flèche correctement
    opacity=0.8
)
Map.add_layer(north_arrow)

#Pour afficher la carte
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

## Superposons ce raster avec un shapefile 

In [9]:
import geemap
import ee

# Initialiser l'API Earth Engine
ee.Initialize() #Cela permet à votre script d'accéder à l'ensemble des fonctionnalités et des données disponibles dans la plateforme Earth Engine.

# Paramètres de visualisation du raster
vis_params = {
    'min': 0,
    'max': 1,
    'palette': ['blue', 'green', 'yellow', 'red']
}


# Charger le shapefile (FeatureCollection)
shapefile = ee.FeatureCollection('projects/ee-khadijaaatou180702/assets/bfa_admbnda_adm_igb_20200323') # Remplacez par le chemin de votre shapefile

# Créer une carte
Map = geemap.Map(center=[0, 0], zoom=2)

# Ajouter le raster à la carte
Map.addLayer(moyenne_raster, vis_params, 'Raster')

# Ajouter le shapefile à la carte
Map.addLayer(shapefile, {'color': 'white'}, 'Shapefile')

# Afficher la carte avec les deux couches superposées
Map


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

## Calcul et visualisation

### Calculons, à présent, les moyennes des pixels par niveau pour avoir les taux de parasite chez les enfants pour le pays, chaque région, chaque province et chaque département


In [35]:
import geemap
import ee

# Charger les shapefiles des régions, provinces et du pays
pays = ee.FeatureCollection('projects/ee-khadijaaatou180702/assets/bfa_admbnda_adm0_igb_20200323')  
regions = ee.FeatureCollection('projects/ee-khadijaaatou180702/assets/bfa_admbnda_adm1_igb_20200323')  
provinces = ee.FeatureCollection('projects/ee-khadijaaatou180702/assets/bfa_admbnda_adm2_igb_20200323')  
departements = ee.FeatureCollection('projects/ee-khadijaaatou180702/assets/bfa_admbnda_adm3_igb_20200323') 

# Fonction pour calculer les moyennes par zone
def statistiques_par_zone(feature_collection, image, statistic_type='mean'):
    stats = image.reduceRegions(
        collection=feature_collection,
        reducer=ee.Reducer.mean(),
        scale=30
    )
    return stats


# Calculer la moyenne pour l'ensemble du pays
moyenne_pays = statistiques_par_zone(pays, moyenne_raster)
pays_result = moyenne_pays.getInfo()

# Calculer la moyenne par province
moyenne_provinces = statistiques_par_zone(provinces, moyenne_raster)
provinces_result = moyenne_provinces.getInfo()

# Calculer la moyenne par région
moyenne_regions = statistiques_par_zone(regions, moyenne_raster)
regions_result = moyenne_regions.getInfo()


# Calculer la moyenne par commune
moyenne_departement = statistiques_par_zone(departements, moyenne_raster)
departements_result = moyenne_departement.getInfo()




Nous allons calculer ces moyennes et les mettre dans des tableaux exportables en fichier csv

In [42]:
import pandas as pd

# Initialiser une liste pour stocker les données
data = []

# Boucler sur les résultats du pays 
for feature in pays_result['features']:
    nom_pays = feature['properties']['ADM0_FR']
    moyenne_valeur = feature['properties']['mean']
    
    # Ajouter chaque ligne sous forme de dictionnaire
    data.append({"Pays": nom_pays, "Moyenne des taux de parasite chez les enfants": moyenne_valeur})

# Convertir la liste en DataFrame
df = pd.DataFrame(data)

# Afficher le tableau (optionnel)
print(df)

# Exporter en CSV
df.to_csv("C:/Users/HP/OneDrive/1231116193333-Desktop/Desktop/Stat_explo_spatiale/Statistique-Exploratoire-Spatiale/TP3/moyenne_par_pays.csv", index=False)


           Pays  Moyenne des taux de parasite chez les enfants
0  Burkina Faso                                       0.551528


In [41]:
import pandas as pd

# Initialiser une liste pour stocker les données
data = []

# Boucler sur les résultats des régions (similaire à `regions_result['features']`)
for feature in regions_result['features']:
    nom_region = feature['properties']['ADM1_FR']
    moyenne_valeur = feature['properties']['mean']
    
    # Ajouter chaque ligne sous forme de dictionnaire
    data.append({"Région": nom_region, "Moyenne des taux de parasite chez les enfants": moyenne_valeur})

# Convertir la liste en DataFrame
df = pd.DataFrame(data)

# Afficher le tableau (optionnel)
print(df)

# Exporter en CSV
df.to_csv("C:/Users/HP/OneDrive/1231116193333-Desktop/Desktop/Stat_explo_spatiale/Statistique-Exploratoire-Spatiale/TP3/moyennes_par_region.csv", index=False)


               Région  Moyenne des taux de parasite chez les enfants
0   Boucle du Mouhoun                                       0.588348
1            Cascades                                       0.555316
2              Centre                                       0.381428
3          Centre-Est                                       0.514621
4         Centre-Nord                                       0.556379
5        Centre-Ouest                                       0.581839
6          Centre-Sud                                       0.500188
7                 Est                                       0.576530
8       Hauts-Bassins                                       0.533363
9                Nord                                       0.550932
10    Plateau-Central                                       0.472275
11              Sahel                                       0.518248
12          Sud-Ouest                                       0.595293


In [39]:
import pandas as pd

# Initialiser une liste pour stocker les données
data = []

# Boucler sur les résultats des provinces 
for feature in provinces_result['features']:
    nom_province = feature['properties']['ADM2_FR']
    moyenne_valeur = feature['properties']['mean']
    
    # Ajouter chaque ligne sous forme de dictionnaire
    data.append({"Province": nom_province, "Moyenne des taux de parasite chez les enfants": moyenne_valeur})

# Convertir la liste en DataFrame
df = pd.DataFrame(data)

# Afficher le tableau (optionnel)
print(df)

# Exporter en CSV
df.to_csv("C:/Users/HP/OneDrive/1231116193333-Desktop/Desktop/Stat_explo_spatiale/Statistique-Exploratoire-Spatiale/TP3/moyennes_par_province.csv", index=False)


       Province  Moyenne des taux de parasite chez les enfants
0          Balé                                       0.585731
1         Banwa                                       0.572779
2         Kossi                                       0.604405
3       Mouhoun                                       0.580954
4        Nayala                                       0.606472
5        Sourou                                       0.582945
6         Comoé                                       0.568416
7        Léraba                                       0.482906
8       Kadiogo                                       0.381428
9       Boulgou                                       0.475284
10   Koulpélogo                                       0.566808
11  Kourittenga                                       0.504774
12          Bam                                       0.571812
13   Namentenga                                       0.584335
14   Sanmatenga                                       0

In [40]:
import pandas as pd
# Initialiser une liste pour stocker les données
data = []

# Supposons que `departements_result` soit le résultat du code initial
for feature in departements_result['features']:
    nom_commune = feature['properties']['ADM3_FR']
    moyenne_valeur = feature['properties']['mean']
    # Ajouter chaque ligne sous forme de dictionnaire
    data.append({"departement": nom_commune, "Moyenne des taux de parasite chez les enfants": moyenne_valeur})

# Convertir en DataFrame
df = pd.DataFrame(data)

# Afficher le tableau
print(df)

# Optionnel : Exporter en CSV
df.to_csv("C:/Users/HP/OneDrive/1231116193333-Desktop/Desktop/Stat_explo_spatiale/Statistique-Exploratoire-Spatiale/TP3/moyennes_par_departement.csv", index=False)





     departement  Moyenne des taux de parasite chez les enfants
0    Komki-Ipala                                       0.469569
1       Komsilga                                       0.419024
2         Koubri                                       0.360350
3    Ouagadougou                                       0.311628
4          Pabré                                       0.441001
..           ...                                            ...
346       Kampti                                       0.601319
347     Loropéni                                       0.623214
348        Malba                                       0.583591
349         Nako                                       0.597619
350     Périgban                                       0.614309

[351 rows x 2 columns]


### Visualisons les cartes avec OpenstreetMap

In [44]:
import geemap
import ee

# Initialiser l'API Earth Engine
ee.Initialize()

# Fonction pour mettre à jour le shapefile avec les moyennes calculées
def shp_avec_les_moyennes(feature_collection, image, mean_property_name='Moyenne_Pixels'):
    stats = image.reduceRegions(
        collection=feature_collection,
        reducer=ee.Reducer.mean(),
        scale=30
    )
    # Ajouter les moyennes comme propriété
    return stats.map(lambda feature: feature.set(mean_property_name, feature.get('mean')))

# Fonction pour créer une image à partir des valeurs moyennes
def creer_image(feature_collection, mean_property_name):
    """Crée une image raster à partir des valeurs moyennes des entités."""
    return feature_collection.reduceToImage(
        properties=[mean_property_name],
        reducer=ee.Reducer.first()
    )

# Mettre à jour les shapefiles avec les moyennes
pays_updated = shp_avec_les_moyennes(pays, moyenne_raster, 'Moyenne_Pays')
regions_updated = shp_avec_les_moyennes(regions, moyenne_raster, 'Moyenne_Région')
provinces_updated = shp_avec_les_moyennes(provinces, moyenne_raster, 'Moyenne_Province')
departements_updated = shp_avec_les_moyennes(departements, moyenne_raster, 'Moyenne_Commune')

# Créer des images pour les moyennes de chaque niveau
pays_image = creer_image(pays_updated, 'Moyenne_Pays')
regions_image = creer_image(regions_updated, 'Moyenne_Région')
provinces_image = creer_image(provinces_updated, 'Moyenne_Province')
departements_image = creer_image(departements_updated, 'Moyenne_Commune')

# Calculer les valeurs min et max pour les visualisations dynamiques
min_value = 0
max_value = 1  

# Paramètres de visualisation dynamiques pour les couleurs
vis_params = {
    'min': min_value,
    'max': max_value,
    'palette': ['blue', 'green', 'yellow', 'red']
}

# Créer une carte pour chaque niveau administratif
Map_pays = geemap.Map(center=[12, -1], zoom=6)
Map_regions = geemap.Map(center=[12, -1], zoom=6)
Map_provinces = geemap.Map(center=[12, -1], zoom=6)
Map_departements = geemap.Map(center=[12, -1], zoom=6)

# Ajouter les images à chaque carte avec les paramètres de visualisation
Map_pays.addLayer(pays_image, vis_params, 'Pays - Moyenne')
Map_regions.addLayer(regions_image, vis_params, 'Régions - Moyenne')
Map_provinces.addLayer(provinces_image, vis_params, 'Provinces - Moyenne')
Map_departements.addLayer(departements_image, vis_params, 'departements - Moyenne')

# Ajouter une légende pour chaque carte
Map_pays.add_colorbar(vis_params=vis_params, label="Moyenne des taux de parasite chez les enfants - Pays")
Map_regions.add_colorbar(vis_params=vis_params, label="Moyenne des taux de parasite chez les enfants - Régions")
Map_provinces.add_colorbar(vis_params=vis_params, label="Moyenne des taux de parasite chez les enfants - Provinces")
Map_departements.add_colorbar(vis_params=vis_params, label="Moyenne des taux de parasite chez les enfants - departements")




Afficher les cartes

In [45]:
print("Carte pour le niveau Pays :")
Map_pays

Carte pour le niveau Pays :


Map(center=[12, -1], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chil…

In [46]:
Map_regions.addLayer(regions, {'color': 'white'}, 'Shapefile')
print("Carte pour le niveau Régions :")
Map_regions

Carte pour le niveau Régions :


Map(center=[12, -1], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chil…

In [47]:
Map_provinces.addLayer(provinces, {'color': 'white'}, 'Shapefile')
print("Carte pour le niveau Provinces :")
Map_provinces

Carte pour le niveau Provinces :


Map(center=[12, -1], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chil…

In [48]:
Map_departements.addLayer(departements, {'color': 'white'}, 'Shapefile')
print("Carte pour le niveau Départements :")
Map_departements

Carte pour le niveau Départements :


Map(center=[12, -1], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chil…

## Affichage des labels

### Essayons now d'afficher les noms des niveaux et les taux moyens correspondants

#### Pour le Burkina Faso

In [54]:
# Paramètres de visualisation
vis_params = {
    'min': min_value,
    'max': max_value,
    'palette': ['pink']
}
# Créer la carte pour les pays
Map_pays = geemap.Map(center=[12, -1], zoom=6)
Map_pays.addLayer(pays_image, vis_params, 'Pays - Moyenne')

# Ajouter les labels pour les pays
for feature in pays_updated.getInfo()['features']:
    pays_name = feature['properties']['ADM0_FR']
    moyenne_valeur = feature['properties']['Moyenne_Pays']
    label_text = f"{pays_name}: {moyenne_valeur:.2f}"
    
    # Extraire les coordonnées du centre du pays
    centroid = ee.Feature(feature).geometry().centroid().coordinates().getInfo()
    
    # Ajouter le texte sur la carte
    Map_pays.add_text(
        text=label_text,
        xy=centroid,
        font_size=15,
        font_color='black',
        font_family='monospace'
    )

# Ajouter une légende pour les pays
Map_pays.add_colorbar(vis_params=vis_params, label="Moyenne des taux de parasite chez les enfants - Pays")


print("Carte pour le Burkina Faso :")
Map_pays

Carte pour le Burkina Faso :


Map(center=[12, -1], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chil…

#### Pour les régions du Burkina Faso

In [53]:
import geemap
import ee

# Initialiser l'API Earth Engine
ee.Initialize()

# Mettre à jour les shapefiles avec les moyennes calculées
def update_shapefile_with_mean(feature_collection, image, mean_property_name='Moyenne_Pixels'):
    stats = image.reduceRegions(
        collection=feature_collection,
        reducer=ee.Reducer.mean(),
        scale=30
    )
    # Ajouter les moyennes comme propriété
    return stats.map(lambda feature: feature.set(mean_property_name, feature.get('mean')))

# Mettre à jour les entités géographiques avec les moyennes des régions
regions_updated = update_shapefile_with_mean(regions, moyenne_raster, 'Moyenne_Région')

# Créer une image des moyennes des régions
regions_image = regions_updated.reduceToImage(
    properties=['Moyenne_Région'],
    reducer=ee.Reducer.first()
)

# Paramètres de visualisation
vis_params = {
    'min': min_value,
    'max': max_value,
    'palette': ['white', '#fee5d9', '#fcae91', '#fb6a4a', '#de2d26', '#a50f15']
}

# Créer la carte
Map_regions = geemap.Map(center=[12, -1], zoom=6)

# Ajouter l'image des régions à la carte
Map_regions.addLayer(regions_image, vis_params, 'Régions - Moyenne')

# Ajouter les labels avec le texte des noms des régions et les moyennes
for feature in regions_updated.getInfo()['features']:
    region_name = feature['properties']['ADM1_FR']
    moyenne_valeur = feature['properties']['Moyenne_Région']
    label_text = f"{region_name}: {moyenne_valeur:.2f}"
    
    # Extraire les coordonnées du centre de chaque région
    centroid = ee.Feature(feature).geometry().centroid().coordinates().getInfo()
    
    # Ajouter le texte sur la carte
    Map_regions.add_text(
        text=label_text,
        xy=centroid,
        font_size=15,
        font_color='black',
        font_family='monospace'
    )

# Ajouter une légende pour la carte
Map_regions.add_colorbar(vis_params=vis_params, label="Moyenne des taux de parasite chez les enfants - Régions")

# Afficher la carte
Map_regions


Map(center=[12, -1], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chil…

#### Pour les provinces du Burkina Faso

In [76]:
import geemap
import ee

# Initialiser l'API Earth Engine
ee.Initialize()

# Mettre à jour les shapefiles avec les moyennes calculées
def update_shapefile_with_mean(feature_collection, image, mean_property_name='Moyenne_Pixels'):
    stats = image.reduceRegions(
        collection=feature_collection,
        reducer=ee.Reducer.mean(),
        scale=30
    )
    # Ajouter les moyennes comme propriété
    return stats.map(lambda feature: feature.set(mean_property_name, feature.get('mean')))

# Mettre à jour les entités géographiques avec les moyennes pour les provinces
provinces_updated = update_shapefile_with_mean(provinces, moyenne_raster, 'Moyenne_Province')

# Créer une image des moyennes pour les provinces
provinces_image = provinces_updated.reduceToImage(
    properties=['Moyenne_Province'],
    reducer=ee.Reducer.first()
)

# Paramètres de visualisation
vis_params = {
    'min': min_value,
    'max': max_value,
    'palette': ['white', '#fee5d9', '#fcae91', '#fb6a4a', '#de2d26', '#a50f15']
}

# Créer la carte pour les provinces
Map_provinces = geemap.Map(center=[12, -1], zoom=6)
Map_provinces.addLayer(provinces_image, vis_params, 'Provinces - Moyenne')

# Ajouter les labels pour les provinces
for feature in provinces_updated.getInfo()['features']:
    province_name = feature['properties']['ADM2_FR']
    moyenne_valeur = feature['properties']['Moyenne_Province']
    label_text = f"{province_name}: {moyenne_valeur:.2f}"
    
    # Extraire les coordonnées du centre de chaque province
    centroid = ee.Feature(feature).geometry().centroid().coordinates().getInfo()
    
    # Ajouter le texte sur la carte
    Map_provinces.add_text(
        text=label_text,
        xy=centroid,
        font_size=12,
        font_color='black',
        font_family='monospace'
    )

# Ajouter une légende pour les provinces
Map_provinces.add_colorbar(vis_params=vis_params, label="Moyenne des taux de parasite chez les enfants - Provinces")

# Afficher la carte pour les provinces
Map_provinces


Map(center=[12, -1], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chil…

#### Pour les départements

In [ ]:
import geemap
import ee

# Initialiser l'API Earth Engine
ee.Initialize()

# Mettre à jour les shapefiles avec les moyennes calculées
def update_shapefile_with_mean(feature_collection, image, mean_property_name='Moyenne_Pixels'):
    stats = image.reduceRegions(
        collection=feature_collection,
        reducer=ee.Reducer.mean(),
        scale=30
    )
    # Ajouter les moyennes comme propriété
    return stats.map(lambda feature: feature.set(mean_property_name, feature.get('mean')))

# Mettre à jour les entités géographiques avec les moyennes pour les departements
departements_updated = update_shapefile_with_mean(departements, moyenne_raster, 'Moyenne_Commune')

# Vérifier les résultats
departements_info = departements_updated.getInfo()
print("Exemple de données mises à jour pour les departements :")
print(departements_info['features'][0]['properties'])  # Imprimer les propriétés d'une des departements pour vérification

# Créer une image des moyennes pour les departements
departements_image = departements_updated.reduceToImage(
    properties=['Moyenne_Commune'],
    reducer=ee.Reducer.first()
)

# Paramètres de visualisation
vis_params = {
    'min': min_value,
    'max': max_value,
    'palette': ['white', '#fee5d9', '#fcae91', '#fb6a4a', '#de2d26', '#a50f15']
}

# Créer la carte pour les departements
Map_departements = geemap.Map(center=[12, -1], zoom=6)
Map_departements.addLayer(departements_image, vis_params, 'departements - Moyenne')

# Ajouter les labels pour les departements
for feature in departements_info['features']:
    commune_name = feature['properties']['ADM3_FR']
    moyenne_valeur = feature['properties'].get('Moyenne_Commune', 'Non calculée')  # Utilisez get() pour éviter KeyError
    label_text = f"{commune_name}: {moyenne_valeur:.2f}" if moyenne_valeur != 'Non calculée' else f"{commune_name}: Non calculée"
    
    # Extraire les coordonnées du centre de chaque commune
    centroid = ee.Feature(feature).geometry().centroid().coordinates().getInfo()
    
    # Ajouter le texte sur la carte
    Map_departements.add_text(
        text=label_text,
        xy=centroid,
        font_size=12,
        font_color='black',
        font_family='monospace'
    )

# Ajouter une légende pour les departements
Map_departements.add_colorbar(vis_params=vis_params, label="Moyenne des taux de parasite chez les enfants - departements")

# Afficher la carte pour les departements
Map_departements
